In [ ]:
import h5py
import numpy as np
import os
import pickle
import torch
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

In [ ]:
def compute_order(data, results):
    segment = torch.from_numpy(data['segment'])[:, :, None, ..., None]
    scatter_shape = [*segment.shape[:2], segment.max() + 1, *segment.shape[3:]]
    obj_mask_true = torch.zeros(scatter_shape).scatter_(2, segment, 1).numpy().astype(np.float64)[:, :, :-1]
    obj_shp_true = data['masks'][:, :, :-1]
    binary_mat_true = np.zeros((*obj_shp_true.shape[:2], obj_shp_true.shape[2], obj_shp_true.shape[2]))
    for i in range(obj_shp_true.shape[2] - 1):
        for j in range(i + 1, obj_shp_true.shape[2]):
            region = np.minimum(obj_shp_true[:, :, i], obj_shp_true[:, :, j])
            area_i = (obj_mask_true[:, :, i] * region).reshape(*region.shape[:2], -1).sum(-1)
            area_j = (obj_mask_true[:, :, j] * region).reshape(*region.shape[:2], -1).sum(-1)
            binary_mat_true[:, :, i, j] = (area_i >= area_j) * 2 - 1
    obj_mask_all = results['mask'][:, :, :, :-1]
    order_cost_all = -(obj_mask_true[None, :, :, :, None] * obj_mask_all[:, :, :, None])
    order_cost_all = order_cost_all.reshape(*order_cost_all.shape[:-3], -1).sum(-1)
    order_cost_all = order_cost_all.sum(2)
    order_all = []
    for cost_list in order_cost_all:
        order_list = []
        for cost in cost_list:
            _, cols = linear_sum_assignment(cost)
            order_list.append(cols)
        order_all.append(order_list)
    order_all = np.array(order_all)
    return order_all, binary_mat_true

def compute_oca(data, results):
    segment = data['segment']
    seg_back = segment.max()
    count_true = np.array([(np.unique(val) != seg_back).sum() for val in segment])
    pres_all = results['pres']
    count_pred = (pres_all[..., :-1] >= 0.5).sum(-1)
    oca_all = (count_pred == count_true[None]).astype(np.float64)
    oca_all = oca_all.mean(-1)
    return oca_all

def compute_ooa(data, results, order_all, binary_mat_true):
    obj_shp_true = data['masks'][:, :, :-1]
    weights = np.zeros((*obj_shp_true.shape[:2], obj_shp_true.shape[2], obj_shp_true.shape[2]))
    for i in range(obj_shp_true.shape[2] - 1):
        for j in range(i + 1, obj_shp_true.shape[2]):
            region = np.minimum(obj_shp_true[:, :, i], obj_shp_true[:, :, j])
            weights[:, :, i, j] = region.reshape(*region.shape[:2], -1).sum(-1)
    sum_weights = weights.sum()
    log_ord_all = results['log_ord']
    ooa_all = []
    for order, log_ord in zip(order_all, log_ord_all):
        binary_mat_pred = np.zeros(weights.shape)
        for idx_data in range(weights.shape[0]):
            sub_order = order[idx_data]
            sub_log_ord = log_ord[idx_data]
            for idx_view in range(weights.shape[1]):
                sub_sub_log_ord = sub_log_ord[idx_view]
                for i in range(order.shape[1] - 1):
                    idx_i = sub_order[i]
                    for j in range(i + 1, order.shape[1]):
                        idx_j = sub_order[j]
                        binary_mat_pred[idx_data, idx_view, i, j] = (sub_sub_log_ord[idx_i] > sub_sub_log_ord[idx_j]) * 2 - 1
        binary_mat = (binary_mat_true * binary_mat_pred) == 1
        ooa_all.append((binary_mat * weights).sum() / sum_weights)
    ooa_all = np.array(ooa_all)
    return ooa_all

def compute_ari_ami(data, results):
    segment_true = data['segment']
    overlap = data['overlap']
    segment_sel = overlap >= 1
    mask_all = results['mask']
    outputs = {key: [] for key in ['ari_all', 'ari_obj', 'ami_all', 'ami_obj']}
    for mask in mask_all:
        segment_a = np.argmax(mask, axis=2).squeeze(-1)
        segment_o = np.argmax(mask[:, :, :-1], axis=2).squeeze(-1)
        sub_outputs = {key: [] for key in outputs}
        for seg_true, seg_sel, seg_a, seg_o in zip(segment_true, segment_sel, segment_a, segment_o):
            seg_a_true_sel = seg_true.reshape(-1)
            seg_o_true_sel = seg_true[seg_sel]
            seg_a_sel = seg_a.reshape(-1)
            seg_o_sel = seg_o[seg_sel]
            sub_outputs['ari_all'].append(adjusted_rand_score(seg_a_true_sel, seg_a_sel))
            sub_outputs['ari_obj'].append(adjusted_rand_score(seg_o_true_sel, seg_o_sel))
            sub_outputs['ami_all'].append(adjusted_mutual_info_score(seg_a_true_sel, seg_a_sel, average_method='arithmetic'))
            sub_outputs['ami_obj'].append(adjusted_mutual_info_score(seg_o_true_sel, seg_o_sel, average_method='arithmetic'))
        for key, val in sub_outputs.items():
            outputs[key].append(val)
    outputs = {key: np.array(val).mean(-1) for key, val in outputs.items()}
    return outputs

def select_by_index(x, index_raw):
    x = torch.from_numpy(x)
    index = torch.from_numpy(index_raw)
    x_ndim = x.ndim
    index_ndim = index.ndim
    index = index.reshape(list(index.shape) + [1] * (x_ndim - index_ndim))
    index = index.expand([-1] * index_ndim + list(x.shape[index_ndim:]))
    if index_raw.ndim == 2:
        x_obj = torch.gather(x[:, :-1], index_ndim - 1, index)
        x = torch.cat([x_obj, x[:, -1:]], dim=1)
    elif index_raw.ndim == 3:
        x_obj = torch.gather(x[:, :, :-1], index_ndim - 1, index)
        x = torch.cat([x_obj, x[:, :, -1:]], dim=2)
    elif index_raw.ndim == 4:
        x_obj = torch.gather(x[:, :, :, :-1], index_ndim - 1, index)
        x = torch.cat([x_obj, x[:, :, :, -1:]], dim=3)
    else:
        raise AssertionError
    return x.numpy()

def compute_iou_f1(data, results, order_all, eps=1e-6):
    obj_shp_true = data['masks'][:, :, :-1]
    order_all = order_all[:, :, None].repeat(obj_shp_true.shape[1], axis=2)
    obj_shp_all = select_by_index(results['shp'], order_all)[:, :, :, :-1]
    seg_true = obj_shp_true.reshape(*obj_shp_true.shape[:3], -1)
    pres = (seg_true.max(-1).max(1) != 0).astype(np.float64)
    sum_pres = pres.sum()
    outputs = {key: [] for key in ['iou', 'f1']}
    for obj_shp in obj_shp_all:
        seg_pred = obj_shp.reshape(*obj_shp.shape[:3], -1)
        area_i = np.minimum(seg_true, seg_pred).sum(-1).sum(1)
        area_u = np.maximum(seg_true, seg_pred).sum(-1).sum(1)
        iou = area_i / np.clip(area_u, eps, None)
        f1 = 2 * area_i / np.clip(area_i + area_u, eps, None)
        outputs['iou'].append((iou * pres).sum() / sum_pres)
        outputs['f1'].append((f1 * pres).sum() / sum_pres)
    outputs = {key: np.array(val) for key, val in outputs.items()}
    return outputs

folder_out = 'outputs'
folder_data = '../ocloc-data'
name_list = ['clevr', 'shop', 'gso', 'shapenet']
phase_list = ['test', 'general']

num_views = 8
metrics = {}
for name_data in name_list:
    metrics[name_data] = {}
    for phase in phase_list:
        metrics[name_data][phase] = {}
        with h5py.File(os.path.join(folder_data, f'{name_data}.h5'), 'r') as f:
            data = {key: f[phase][key][:, :num_views] for key in f[phase]}
            for key, val in data.items():
                if key in ['segment', 'overlap']:
                    data[key] = val.astype(np.int64)
                else:
                    data[key] = val.astype(np.float64) / 255
        with h5py.File(os.path.join(folder_out, f'{name_data}_multi', f'{phase}.h5'), 'r') as f:
            results = {key: f[key][()] / 255 for key in f}
        results['log_ord'] = results['log_ord'].squeeze(-1)
        order_all, binary_mat_true = compute_order(data, results)
        metrics[name_data][phase]['oca'] = compute_oca(data, results)
        metrics[name_data][phase]['ooa'] = compute_ooa(data, results, order_all, binary_mat_true)
        metrics[name_data][phase].update(compute_ari_ami(data, results))
        metrics[name_data][phase].update(compute_iou_f1(data, results, order_all))
    with open('metrics.pkl', 'wb') as f:
        pickle.dump(metrics, f)

In [ ]:
with open('metrics.pkl', 'rb') as f:
    metrics = pickle.load(f)
for name_data in metrics:
    print(name_data)
    for phase in metrics[name_data]:
        print(phase)
        for key_list in [['ari_all', 'ami_all', 'ari_obj', 'ami_obj'], ['iou', 'f1', 'oca', 'ooa']]:
            text_list = []
            for key in key_list:
                val = metrics[name_data][phase][key]
                if val is None:
                    text_list.append(f'{key:<7}: {"N/A":<11}')
                else:
                    text_list.append(f'{key:<7}: {val.mean():.3f}' + '\u00b1' + f'{val.std():.0e}')
            text = (' ' * 8).join(text_list)
            print(text)
    print()